# Setup

In [ ]:
path_booking = '/content/drive/MyDrive/HUST/20202/Tích hợp dữ liệu/booking.com/booking.json'
path_hanoi = '/content/drive/MyDrive/HUST/20202/Tích hợp dữ liệu/data/traveloka/ha_noi.csv'

# Matcher

In [ ]:
#Xây dựng và tối ưu tham số Matcher
from recordlinkage.compare import Geographic

compare = recordlinkage.Compare()
compare.add(Geographic('lot','lat','lot','lat',offset=5,scale=2, method='gauss',label='distance'))


compare.string('new_add',
             'new_add',
             method='jarowinkler',
              threshold=0.75,
            
           label='add')

compare.string('new_name',
             'new_name',
             method='levenshtein',
              threshold=0.8,
            
           label='name')


In [ ]:
#Lọc trùng
indexer = recordlinkage.Index()
indexer.block(left_on='city')

eb_dup=indexer.index(ebooking)
eb_ft=compare.compute(eb_dup,ebooking)

In [ ]:
eb_ft.sum(axis=1).value_counts().sort_index(ascending=False)

In [ ]:
eb_matches = eb_ft[eb_ft.sum(axis=1) >=2.5].reset_index()
eb_matches

In [ ]:
#Kiểm tra trường còn lại khi khớp tên
eb_matches = eb_ft[eb_ft['name'] ==1].reset_index()

In [ ]:
ebooking.loc[618]

In [ ]:
ebooking.loc[603]

In [ ]:
#ebooking 3 cặp record trùng ==> xóa 3 record
ebooking.drop(ebooking.index[[594,603,488]], inplace=True)
#traveloka 3 cặp record bị trùng ==>xóa 3 record
traveloka.drop(traveloka.index[[432,391,505]], inplace=True)
#booking 3 cặp trùng và 1 bộ 3 trùng nhau ==> xóa 5 record
booking.drop(booking.index[[950,998,218,919,947]], inplace=True)

In [ ]:
agoda['new_name']=agoda['hotel name'].str.lower()
agoda['new_name'].replace(namels,'', inplace=True, regex=True)
agoda['new_name']=agoda['new_name'].fillna('NA')
agoda['new_name']=[unidecode.unidecode(y) for y in agoda['new_name'] ] 
agoda['new_name'] = agoda['new_name'].apply(lambda x: x.partition('(')[2].replace(')','') if x.partition('(')[1]=='(' else x )

# Agoda

In [ ]:
#trường hợp của agoda
agoda.loc[30]
agoda.loc[17]
# ==> hiệu chỉnh lại new_name và chỉnh method của camparer
# chungcu->cu, phongngu-> gu, phongtamrieng -> mr
# namels2=['hungc','phongn','phongta','ieng','nho']
# agoda['new_name'].replace(namels2,'', inplace=True, regex=True)
compare2 = recordlinkage.Compare()
compare2.add(Geographic('lat','lot','lat','lot',offset=1,scale=0.2, method='gauss',label='distance'))
compare2.string('new_add',
             'new_add',
             method='jarowinkler',
              threshold=0.75,
            
           label='add')

compare2.string('new_name',
             'new_name',
             method='jarowinkler',
              threshold=0.8,
            
           label='name')

In [ ]:
eb_matches = eb_ft[eb_ft.sum(axis=1) >=2.5].reset_index()
eb_matches 

In [ ]:
#loại bỏ trùng khách sạn để lại giá phòng thấp nhất
agoda.drop(agoda.index[[189,775,374,289,918,1135,1128,708,1158,325,1040]], inplace=True)

# Matching

In [ ]:
take_list=['new_name','city','new_add','lat','lot']
agox=agoda[take_list]
agox['id2']=agox.index

ebx=ebooking[take_list]
ebx['id3']=ebx.index

bokx=booking[take_list]
bokx['id1']=bokx.index

tvx=traveloka[take_list]
tvx['id4']=tvx.index

In [ ]:
new_data=bokx.append([agox,ebx,tvx],ignore_index=True)

In [ ]:
indexer = recordlinkage.Index()
indexer.block(left_on='city')

eb_dup=indexer.index(new_data)
eb_ft=compare.compute(eb_dup,new_data)

In [ ]:
eb_ft.sum(axis=1).value_counts().sort_index(ascending=False)

In [ ]:
eb_matches = eb_ft[eb_ft['name'].values*6+ eb_ft['add'].values*3+eb_ft['distance'].values >=9.2 ].reset_index()
eb_matches

In [ ]:
new_data.loc[1006]

In [ ]:
new_data.loc[104]

In [ ]:
#tổng hợp dữ liệu
match=eb_matches['level_0'].unique()
num=len(match)
tick=[1]*max(eb_matches['level_0'].max(),eb_matches['level_1'].max())+[1]
ls=[]
for i in match:
    #if tick[i]:
        
        a=set()
        
        b=set()
        b.add(i)
        while b:
            val=b.pop()
            if tick[val]:
                a.add(val)
                tick[val]=0
                x=eb_matches.loc[eb_matches['level_0']==val,'level_1'].to_list()
                x=x+eb_matches.loc[eb_matches['level_1']==val,'level_0'].to_list()
                for j in x:
                    if j:
                        b.add(j)
                        a.add(j)
        if a:
            ls.append(a)

In [ ]:
for i in ls:
    print(i)

In [ ]:
new_rows=pd.DataFrame()
for i in ls:
    new_rows=new_rows.append(new_data.loc[i,:].sum(axis=0),ignore_index=True)

# Make web-data

In [ ]:
#Tạo dataframe cho dữ liệu match:
n_data=pd.DataFrame()
n_data['name']=[]
n_data['img_url']=[]
n_data['url1']=[]
n_data['url2']=[]
n_data['url3']=[]
n_data['url4']=[]

n_data['city']=[]
n_data['address']=[]
n_data['description']=[]
n_data['stars']=[]
n_data['price']=[]
n_data['rating']=[]
    #num=0
n_data['number_review']=[]


n_data['nearby_places']=[]

n_data['facilities']=[]
n_data

In [ ]:


for i in range(len(new_rows)):
    id1=new_rows['id1'][i]
    id2=new_rows['id2'][i]
    id3=new_rows['id3'][i]
    id4=new_rows['id4'][i]
    url1=''
    url2=''
    url3=''
    url4=''
    name=''
    city=''
    stars=''
    price=intmax
    rating=''
    
    number_review=0
    address=''
    description=''
    nearby_places=''
    img_url=''
    facilities=''
    if id1:
        
        try:
            name=booking['hotel name'][id1]
            url1=booking['url'][id1]
            city=booking['city'][id1]
            stars=booking['stars'][id1]
            price=min(price,booking['price'][id1])
            rating=booking['rating'][id1]
            number_review=number_review+booking['n_reviews'][id1]
            address=booking['address'][id1]
            description=booking['description'][id1]
            nearby_places=booking['nearby places'][id1]
            facilities=booking['facilities'][id1] 
            img_url=booking['images'][id1] 
        except:
            pass
    if id2:
        try: 
            name=agoda['hotel name'][id2]
            url2=agoda['url'][id2]
            city=agoda['city'][id2]
            stars=agoda['stars'][id2]
            price=min(price,agoda['price'][id2])
            rating=agoda['rating'][id2]
            number_review=number_review+agoda['n_reviews'][id2]
            address=agoda['address'][id2]

            nearby_places=agoda['nearby places'][id2]
            facilities=agoda['facilities'][id2]
            if not img_url:
                img_url=agoda['image'][id2]
        except:
            pass
    if id4:
        try:
            name=traveloka['name'][id4]
            url4=traveloka['url'][id4]
            city=traveloka['city'][id4]
            stars=traveloka['star'][id4]
            price=min(price,traveloka['price'][id4])
            rating=traveloka['rating'][id4]
            number_review=number_review+traveloka['number_review'][id4]
            address=traveloka['address'][id4]
            description=traveloka['hotel_descriptions'][id4]
            nearby_places=traveloka['nearby_places'][id4]
            facilities=traveloka['facilities'][id4]
        except:
            pass
    if id3:
        try:
            name=ebooking['name'][id3]
            url3=ebooking['url'][id3]
            city=ebooking['city'][id3]
            stars=ebooking['star'][id3]
            price=min(price,ebooking['price'][id3])
            rating=ebooking['rating'][id3]
            number_review=number_review+ebooking['n_review'][id3]
            address=ebooking['address'][id3]

            nearby_places=ebooking['nearby'][id3]
            if not facilities:
                facilities=ebooking['facilities'][id3]
            if not img_url:
                img_url=ebooking['img'][id3]
        except:
            pass
    ind=len(n_data)
    n_data.loc[ind,'name']=name
    n_data.loc[ind,'img_url']=img_url
    n_data.loc[ind,'url1']=url1
    n_data.loc[ind,'url2']=url2
    n_data.loc[ind,'url3']=url3
    n_data.loc[ind,'url4']=url4

    n_data.loc[ind,'city']=city
    n_data.loc[ind,'address']=address
    n_data.loc[ind,'description']=description
    n_data.loc[ind,'stars']=stars
    n_data.loc[ind,'price']=price
    n_data.loc[ind,'rating']=rating
        #num=0
    n_data.loc[ind,'number_review']=number_review


    n_data.loc[ind,'nearby_places']=nearby_places

    n_data.loc[ind,'facilities']=facilities


In [ ]:
n_data['slug']=n_data['name'].str.lower()
n_data['slug']=[unidecode.unidecode(y) for y in n_data['slug'].fillna('NA') ]
n_data['slug'] = [y.partition('(')[0] for y in n_data['slug'] ]
n_data['slug'].replace(' ','_', inplace=True, regex=True)
n_data

# Add non-match data

In [ ]:
drop_booking=new_rows.loc[new_rows['id1'] >0,'id1']
drop_ago=new_rows.loc[new_rows['id2'] >0,'id2']
drop_eb=new_rows.loc[new_rows['id3'] >0,'id3']
drop_tra=new_rows.loc[new_rows['id4'] >0,'id4']